# Data augmentation


Die Daten wurde erweitert durch folgende techniken:
- flipping
- cropping
- rotation
- brightness adjustment


Ich habe dafür die [Albumentations](https://albumentations.ai/docs/) Bibliothek verwendet.

Ich hab mich gegen "coarse dropout" entschieden, weil es unrealistsch ist, denke ich. Man bekommt nicht ein Foto, an dem ein Teil fehlt. Aber vielleicht, ist es Sinnvoll.


In [ ]:
import albumentations as A
from albumentations.augmentations.pixel import functional as F
import numpy as np

def augment(images, masks):
  augmented_images = []
  augmented_masks = []

  for img, msk in zip(images, masks):
    augmented_images.append(img)
    augmented_masks.append(msk)
    
    grid_distortion = A.GridDistortion(p=1.0, distort_limit=0.6)(image=img, mask=msk)
    augmented_images.append(grid_distortion['image'])
    augmented_masks.append(grid_distortion['mask'])
    
    optical_distortion = A.OpticalDistortion(p=1.0, distort_limit=1, mode='camera')(image=img, mask=msk)
    augmented_images.append(optical_distortion['image'])
    augmented_masks.append(optical_distortion['mask'])
    
    elastic_transform = A.ElasticTransform(p=1.0, alpha=120, sigma=120)(image=img, mask=msk)
    augmented_images.append(elastic_transform['image'])
    augmented_masks.append(elastic_transform['mask'])

    flipped = A.HorizontalFlip(p=1.0)(image=img, mask=msk)
    augmented_images.append(flipped['image'])
    augmented_masks.append(flipped['mask'])

    cropped = A.RandomResizedCrop(
      size=(224, 224)
    )(image=img, mask=msk)
    augmented_images.append(cropped['image'])
    augmented_masks.append(cropped['mask'])

  #  randomly_erased = A.CoarseDropout(
  #      hole_height_range=(0.3, 0.5),
  #      hole_width_range=(0.3, 0.5),
  #      fill_mask=0,
  #      p=1.0
  #  )(image=img, mask=msk)
  #  augmented_images.append(randomly_erased['image'])
  #  augmented_masks.append(randomly_erased['mask'])

    rotated = A.RandomRotate90()(image=img, mask=msk)
    augmented_images.append(rotated['image'])
    augmented_masks.append(rotated['mask'])

    brightness = A.RandomBrightnessContrast(p=1.0)(image=img, mask=msk)
    augmented_images.append(brightness['image'])
    augmented_masks.append(brightness['mask'])
    
    gaussian_blur = A.GaussianBlur(p=1.0, sigma_limit=(1, 1.5))(image=img, mask=msk)
    augmented_images.append(gaussian_blur['image'])
    augmented_masks.append(gaussian_blur['mask'])

    
  # Convert lists to NumPy arrays
  augmented_images = np.array(augmented_images)
  augmented_masks = np.array(augmented_masks)
  
  return (augmented_images, augmented_masks)


if images is not None and masks is not None:
  augmented_images, augmented_masks = augment(images, masks)
  